In [214]:
import os
import re
import csv

import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.markers import MarkerStyle
import numpy as np
import math


In [215]:
sm = {}
sm["directory"]= "20230727"
sm["file_includes"] = "_sm_"


p = re.compile(r"\d\d\d")
instance_size_regex = re.compile(r"\d\d\d\d")

GROUNDING_TIME = "grounding_time"
GROUNDING_SIZE = "grounding_size"
TOTAL_TIME = "total_time"

GRINGO = "gringo"
IDLV = "idlv"
NEWGROUND_GRINGO = "newground-gringo"
NEWGROUND_IDLV = "newground-idlv"


grounder_types = [GRINGO, IDLV, NEWGROUND_GRINGO, NEWGROUND_IDLV]


file_types = [GROUNDING_TIME, GROUNDING_SIZE, TOTAL_TIME]

S1 = "S1"

S2 = "S2"

S3_T4 = "S3-T4"
S3_T6 = "S3-T6"
S3_T8 = "S3-T8"

S4_T4 = "S4-T4"
S4_T6 = "S4-T6"
S4_T8 = "S4-T8"



In [216]:
problem_structure = {}

problem_structure[S1] = {}
problem_structure[S1]["directory"]= ["20230727"]
problem_structure[S1]["file_includes"] = "_sm_"

problem_structure[S2] = {}
problem_structure[S2]["directory"]= ["20230804","20230810"]
problem_structure[S2]["file_includes"] = "benchmark_output_nprc_"

problem_structure[S3_T4] = {}
problem_structure[S3_T4]["directory"]= ["20230803"]
problem_structure[S3_T4]["file_includes"] = "_output_tpg_star_4_"

problem_structure[S3_T6] = {}
problem_structure[S3_T6]["directory"]= ["20230803"]
problem_structure[S3_T6]["file_includes"] = "_output_tpg_star_6_"

problem_structure[S3_T8] = {}
problem_structure[S3_T8]["directory"]= ["20230803"]
problem_structure[S3_T8]["file_includes"] = "_output_tpg_star_8_"

problem_structure[S4_T4] = {}
problem_structure[S4_T4]["directory"]= ["20230805"]
problem_structure[S4_T4]["file_includes"] = "_output_tpg_02_star_4_"

problem_structure[S4_T6] = {}
problem_structure[S4_T6]["directory"]= ["20230805"]
problem_structure[S4_T6]["file_includes"] = "_output_tpg_02_star_6_"


problem_structure[S4_T8] = {}
problem_structure[S4_T8]["directory"]= ["20230805"]
problem_structure[S4_T8]["file_includes"] = "_output_tpg_02_star_8_"



In [217]:
label_converter = {}
label_converter[GRINGO] = GRINGO
label_converter[IDLV] = IDLV
label_converter[NEWGROUND_GRINGO] = "NaGG-Gringo"
label_converter[NEWGROUND_IDLV] = "NaGG-IDLV"

In [218]:
type_structure = {}

for problem in problem_structure.keys():
    type_structure[problem] = {}
    type_structure[problem]["instance_size"] = 0
    for grounder_type in grounder_types:
        type_structure[problem][grounder_type] = {}
        for file_type in file_types:
            type_structure[problem][grounder_type][file_type] = {}
            type_structure[problem][grounder_type][file_type]["solved"] = 0
            type_structure[problem][grounder_type][file_type]["solved_list"] = []

            type_structure[problem][grounder_type]["total_" + file_type] = 0
            
for problem in problem_structure.keys():
    type_structure[problem]["timeout"] = 1800
    
type_structure[S1]["instance_size"] = 50
type_structure[S1]["instance_size_p"] = 50

type_structure[S2]["instance_size_p"] = 31
type_structure[S2]["instance_size"] = 31 * 10

type_structure[S3_T4]["instance_size_p"] = 40
type_structure[S3_T6]["instance_size_p"] = 40
type_structure[S3_T8]["instance_size_p"] = 40

type_structure[S3_T4]["instance_size"] = 40 * 10
type_structure[S3_T6]["instance_size"] = 40 * 10
type_structure[S3_T8]["instance_size"] = 40 * 10


type_structure[S4_T4]["instance_size"] = 40 * 10
type_structure[S4_T6]["instance_size"] = 40 * 10
type_structure[S4_T8]["instance_size"] = 40 * 10


type_structure[S4_T4]["instance_size_p"] = 40
type_structure[S4_T6]["instance_size_p"] = 40
type_structure[S4_T8]["instance_size_p"] = 40

type_structure[S2]["timeout"] = 10800

#for problem in problems[-1:]:
#    type_structure[problem]["timeout"] = 10800
    

In [219]:
def read_csv_helper(directory, file, file_type):
    
    dens_structure = {}

    dens_structure["filename"] = file
    dens_structure["type"] = file_type
    file_data = pd.read_csv(os.path.join(directory, file), delimiter=',')    
    dens_structure["csv"] = file_data
    
    return (dens_structure)

In [220]:
def fill_type_structure(file_data, problem, type_structure):

    for column_name in list(file_data.columns):
        for grounder_type in grounder_types:
            if column_name.startswith(grounder_type) and "timeout-occurred" not in column_name and "timeout-occured" not in column_name:
                if "instance_size" not in type_structure[problem][grounder_type][file_type]:
                    type_structure[problem][grounder_type][file_type]["instance_size"] = []
                    
                if "size" in column_name:
                    new_column_name = GROUNDING_SIZE
                elif "grounding" in column_name:
                    new_column_name = GROUNDING_TIME
                else:
                    new_column_name = TOTAL_TIME
                    
                    
                if new_column_name not in type_structure[problem][grounder_type][file_type]:
                    type_structure[problem][grounder_type][file_type][new_column_name] = []


                list_instances = list(file_data["instance"])
                    

                instance_size_list = [int((instance_size_regex.search(instance)).group(0)) for instance in list_instances]


                type_structure[problem][grounder_type][file_type]["instance_size"] += instance_size_list
                
                timeout = type_structure[problem]["timeout"]

                raw_column_data = list(file_data[column_name])
                if "duration" in column_name:
                    parsed_column_data = [time if float(time) < timeout else -1 for time in raw_column_data]
                    
                    total_duration = sum([item if item > 0 else timeout for item in parsed_column_data])
                    type_structure[problem][grounder_type]["total_" + file_type] += total_duration
                    
                elif "size" in column_name:
                    parsed_column_data = [(float(size)/(1024 * 1024)) if float(size) <= 32*(math.pow(10,9)) else -1 for size in raw_column_data]

                    total_size = sum([item if item > 0 else 32*(math.pow(10,3)) for item in parsed_column_data])
                    type_structure[problem][grounder_type]["total_" + file_type] += total_size

                else:
                    print("THIS SHOULD NEVER HAPPEN!")
                    total_time
                solved_list = [1 if i > 0 else 0 for i in parsed_column_data]
                
                instance_size_p = type_structure[problem]["instance_size_p"]
                
                if len(solved_list) < instance_size_p:
                    len_solved_list = len(solved_list)
                    avg_gt_list = [0 for i in range(instance_size_p - len_solved_list)]
                    solved_list = solved_list + avg_gt_list
                    
                    if "duration" in column_name:               
                        parsed_column_data = parsed_column_data + [timeout for i in range(instance_size_p - len_solved_list)]
                    elif "size" in column_name:                        
                        parsed_column_data = parsed_column_data + [32*(math.pow(10,3)) for i in range(instance_size_p - len_solved_list)]
                
                
                solved = sum(solved_list)
                
                type_structure[problem][grounder_type][file_type]["solved"] += solved
                type_structure[problem][grounder_type][file_type]["solved_list"] += solved_list

                
                    
                type_structure[problem][grounder_type][file_type][new_column_name] += parsed_column_data
                



In [221]:
# Parse all files into type_structure
ds = sm

for problem in problem_structure.keys():
    ds = problem_structure[problem]
    
    for directory in ds["directory"]:
        for root, dirs, files in os.walk(directory):
            for file in files:

                if ds["file_includes"] in file and not "log" in file:

                    for file_type in file_types:
                        if file_type in file:
                            csv_dict = read_csv_helper(directory, file, file_type)



                            #if "grounding_size" in file and not "S3" in problem and not "S4" in problem:
                            #    cur_size = type_structure[problem]["instance_size"]
                            #    instances = len(list(csv_dict["csv"]["instance"]))
                            #    type_structure[problem]["instance_size"] += instances

                            #density_structure[star][density][file_type] = local_density_structure

                            fill_type_structure(csv_dict["csv"], problem, type_structure)


In [222]:
#grounded_list = [["grounded", grounder_types[i]] for i in range(4)]
#solved_list = [["solved", grounder_types[i]] for i in range(4)]

rounding_precision = 2

grounded_list = [[] for i in range(4)]
solved_list = [[] for i in range(4)]

totals_times_list = [[] for i in range(4)]
totals_grounding_list = [[] for i in range(4)]
totals_size_list = [[] for i in range(4)]

avg_gt_list = [[] for i in range(4)]
avg_tt_list = [[] for i in range(4)]
avg_gs_list = [[] for i in range(4)]

median_gt_list = [[] for i in range(4)]
median_tt_list = [[] for i in range(4)]
median_gs_list = [[] for i in range(4)]

size_list = []

for problem in problem_structure.keys():
    counter = 0
    
    instance_size = type_structure[problem]["instance_size"]
    timeout = type_structure[problem]["timeout"]

    
    size_list.append(instance_size)
    
    for grounder in grounder_types:
        type_structure[problem][grounder]["total_solved"] = type_structure[problem][grounder][TOTAL_TIME]["solved"]
        
        solved_time = type_structure[problem][grounder][GROUNDING_TIME]["solved"]
        solved_size = type_structure[problem][grounder][GROUNDING_SIZE]["solved"]
        
        solved_time_list = type_structure[problem][grounder][GROUNDING_TIME]["solved_list"]
        solved_size_list = type_structure[problem][grounder][GROUNDING_SIZE]["solved_list"]
        
        new_grounded_time_list = []
        new_grounded_size_list = []
        
        grounded_time_list = type_structure[problem][grounder][GROUNDING_TIME][TOTAL_TIME]
        grounded_size_list = type_structure[problem][grounder][GROUNDING_SIZE][GROUNDING_SIZE]
        
        #print(grounded_size_list)
        
        for i in range(len(solved_time_list)):
            if solved_time_list[i] == 0 or solved_size_list[i] == 0:
                new_grounded_time_list.append(timeout)
                new_grounded_size_list.append(32*(math.pow(10,3)))
            else:
                new_grounded_time_list.append(grounded_time_list[i])
                new_grounded_size_list.append(grounded_size_list[i])
                
        #print(new_grounded_size_list)
        #new_grounded_time_list = type_structure[problem][grounder][GROUNDING_TIME][TOTAL_TIME]
        #new_grounded_size_list = type_structure[problem][grounder][GROUNDING_SIZE][GROUNDING_SIZE]
                
                
        if solved_time < solved_size:
            type_structure[problem][grounder]["grounded"] = solved_time
        else:
            type_structure[problem][grounder]["grounded"] = solved_size
            
        total_solved = type_structure[problem][grounder]["total_solved"]

        total_solved_list = type_structure[problem][grounder][TOTAL_TIME]["solved_list"]
        total_time_list = type_structure[problem][grounder][TOTAL_TIME][TOTAL_TIME]

        new_total_list = []
        
        for i in range(len(total_solved_list)):
            if total_solved_list[i] == 0:
                new_total_list.append(timeout)
            else:
                new_total_list.append(total_time_list[i])
                
        #new_total_list = type_structure[problem][grounder][TOTAL_TIME][TOTAL_TIME]
            
        total_solved_percent = float(total_solved)/instance_size
        
        total_grounded = type_structure[problem][grounder]["grounded"] 
        total_grounded_percent = float(total_grounded)/instance_size
        
        solved_list[counter].append(total_solved)
        grounded_list[counter].append(total_grounded)
        
        #solved_list[counter].append(total_solved_percent)
        #grounded_list[counter].append(total_grounded_percent)

        diff_solved = instance_size - total_solved
        diff_grounded = instance_size - total_grounded
        
        total_time = int(type_structure[problem][grounder]["total_" + TOTAL_TIME]) + int(diff_solved * type_structure[problem]["timeout"])
        total_time = sum(new_total_list)
        total_time = round(float(total_time) / 3600,rounding_precision)
        
        #total_grounding_time = int(type_structure[problem][grounder]["total_" + GROUNDING_TIME]) + int(diff_grounded * type_structure[problem]["timeout"])
        total_grounding_time = sum(new_grounded_time_list)
        total_grounding_time = round(float(total_grounding_time) / 3600,rounding_precision)
        
        
        #total_grounding_size = int(type_structure[problem][grounder]["total_" + GROUNDING_SIZE]) + int(diff_grounded * 32*(math.pow(10,3)))
        total_grounded_size = sum(new_grounded_size_list)
        
        #if problem != S2:
        #    total_grounding_size = round(float(total_grounding_size) / (1024 * 1024), rounding_precision)
        #else:
        #    total_grounding_size = round(float(total_grounding_size), rounding_precision)
        
        totals_times_list[counter].append(total_time)
        
        totals_grounding_list[counter].append(total_grounding_time)
        
        totals_size_list[counter].append(total_grounded_size) 
        

        #list_consideration = [item if item > 0 else timeout for item in type_structure[problem][grounder][GROUNDING_TIME]["total_time"]]
        list_consideration = new_grounded_time_list
        avg_gt_list[counter].append(round(np.average(list_consideration),rounding_precision))
        median_gt_list[counter].append(round(np.median(list_consideration),rounding_precision))
        
        #list_consideration = [item if item > 0 else 32*(math.pow(10,3)) for item in type_structure[problem][grounder][GROUNDING_SIZE]["grounding_size"]]
        list_consideration = new_grounded_size_list
        avg_gs_list[counter].append(round(np.average(list_consideration),rounding_precision))
        median_gs_list[counter].append(round(np.median(list_consideration),rounding_precision))
        
        #list_consideration = [item if item > 0 else timeout for item in type_structure[problem][grounder][TOTAL_TIME]["total_time"]]
        list_consideration = new_total_list
        avg_tt_list[counter].append(round(np.average(list_consideration),rounding_precision))
        median_tt_list[counter].append(round(np.median(list_consideration),rounding_precision))

        counter += 1


        

In [223]:
instance_list = list(problem_structure.keys())

In [224]:
h1 = ["solved" for i in range(4)]
total_times_h1_header = ["total times [h]" for i in range(4)] 
grounding_times_h1_header = ["grounding times [h]" for i in range(4)] 
grounding_sizes_h1_header = ["grounding sizes [MB]" for i in range(4)] 

totals_h2_header = [label_converter[grounder_types[i]] for i in range(4)]
solved_h2 = [label_converter[grounder_types[i]] for i in range(4)]


h1_ = ["instances"] +\
    h1 +\
    grounding_times_h1_header +\
    grounding_sizes_h1_header
    
    
h2_ = ["instances"] + solved_h2 + (totals_h2_header * 2)

columns = [h1_,h2_]

#frame = pd.DataFrame(list(zip(instance_list, size_list, *grounded_list, *solved_list)))

frame = pd.DataFrame(list(zip(instance_list, size_list, *solved_list, *totals_grounding_list, *totals_size_list)))
frame.columns = ["instance"] + h2_
frame = frame.set_index("instance")
frame.columns = columns


In [225]:
frame

instances solved                            grounding times [h]   
         instances gringo idlv NaGG-Gringo NaGG-IDLV              gringo   
instance                                                                   
S1              50     26   50          50        50                7.54  \
S2             310     27   93          27       100             1723.03   
S3-T4          400    116  137         214       215              138.91   
S3-T6          400     65   74         198       197              164.91   
S3-T8          400     49   50         175       170              175.54   
S4-T4          400     48   77         122       122              117.27   
S4-T6          400     19   32          68        66               92.35   
S4-T8          400     13   20          56        57               93.89   

                                        grounding sizes [MB]                 
             idlv NaGG-Gringo NaGG-IDLV               gringo          idlv   
instance                                                                     
S1           3.59        2.03      1.49         3.989168e+02  1.848627e+02  \
S2        1262.46     1729.16   1267.46         1.833618e+07  1.328098e+07   
S3-T4      143.46      105.21    105.01         8.640171e+06  8.960225e+06   
S3-T6      163.47      117.90    117.61         1.040006e+07  1.027223e+07   
S3-T8      175.08      130.83    130.82         1.110407e+07  1.110419e+07   
S4-T4      106.65       91.07     91.13         7.456092e+06  6.656166e+06   
S4-T6       85.23       72.78     72.74         5.888051e+06  5.376067e+06   
S4-T8       91.27       76.28     76.28         5.984042e+06  5.760055e+06   

                                      
           NaGG-Gringo     NaGG-IDLV  
instance                              
S1        1.684743e+01  7.471056e-01  
S2        1.840013e+07  1.270499e+07  
S3-T4     5.888001e+06  5.824002e+06  
S3-T6     6.400001e+06  6.432001e+06  
S3-T8     7.136001e+06  7.168001e+06  
S4-T4     5.056000e+06  5.056000e+06  
S4-T6     4.224000e+06  4.256000e+06  
S4-T8     4.608000e+06  4.544000e+06

In [226]:
my_latex = frame.to_latex(float_format="%.1f", column_format = "lrrrrrrrrrrrrr")
#my_latex = frame.to_latex()

print(my_latex)

\begin{tabular}{lrrrrrrrrrrrrr}
\toprule
 & instances & \multicolumn{4}{r}{solved} & \multicolumn{4}{r}{grounding times [h]} & \multicolumn{4}{r}{grounding sizes [MB]} \\
 & instances & gringo & idlv & NaGG-Gringo & NaGG-IDLV & gringo & idlv & NaGG-Gringo & NaGG-IDLV & gringo & idlv & NaGG-Gringo & NaGG-IDLV \\
instance &  &  &  &  &  &  &  &  &  &  &  &  &  \\
\midrule
S1 & 50 & 26 & 50 & 50 & 50 & 7.5 & 3.6 & 2.0 & 1.5 & 398.9 & 184.9 & 16.8 & 0.7 \\
S2 & 310 & 27 & 93 & 27 & 100 & 1723.0 & 1262.5 & 1729.2 & 1267.5 & 18336177.3 & 13280980.9 & 18400131.3 & 12704988.9 \\
S3-T4 & 400 & 116 & 137 & 214 & 215 & 138.9 & 143.5 & 105.2 & 105.0 & 8640171.3 & 8960225.0 & 5888001.4 & 5824001.6 \\
S3-T6 & 400 & 65 & 74 & 198 & 197 & 164.9 & 163.5 & 117.9 & 117.6 & 10400064.3 & 10272229.8 & 6400001.3 & 6432001.4 \\
S3-T8 & 400 & 49 & 50 & 175 & 170 & 175.5 & 175.1 & 130.8 & 130.8 & 11104069.9 & 11104189.9 & 7136001.0 & 7168001.1 \\
S4-T4 & 400 & 48 & 77 & 122 & 122 & 117.3 & 106.7 & 91.1 & 91.1 &

In [223]:
h1 = ["solved (\%)" for i in range(4)]
avg_total_times_h1_header = ["avg total times (s)" for i in range(4)] 
avg_grounding_times_h1_header = ["avg grounding times (s)" for i in range(4)] 
avg_grounding_sizes_h1_header = ["avg grounding sizes (KiB)" for i in range(4)] 

totals_h2_header = [label_converter[grounder_types[i]] for i in range(4)]
solved_h2 = [label_converter[grounder_types[i]] for i in range(4)]

h1_ = ["instances"] + avg_total_times_h1_header +\
    avg_grounding_times_h1_header +\
    avg_grounding_sizes_h1_header +\
    h1
    
h2_ = ["instances"] + (totals_h2_header * 3) + solved_h2

columns = [h1_,h2_]

frame = pd.DataFrame(list(zip(instance_list, size_list, *avg_tt_list, *avg_gt_list, *avg_gs_list,  *solved_list)))
frame.columns = ["instance"] + h2_
frame = frame.set_index("instance")
frame.columns = columns

In [208]:
frame

instances avg total times (s)                                  
         instances              gringo     idlv NaGG-Gringo NaGG-IDLV   
instance                                                                
S1              50             1192.70   447.38      200.51    121.79  \
S2             270             9730.77  7136.08     9732.72   6991.28   
S3-T4          400             1120.29  1041.48      626.94    638.34   
S3-T6          400             1345.08  1313.57      621.14    615.99   
S3-T8          400             1450.49  1426.33      700.61    721.38   
S4-T4          400             1268.23   997.33      673.07    673.72   
S4-T6          400             1472.54  1241.39      747.33    746.02   
S4-T8          400             1551.76  1443.47      797.64    797.32   

         avg grounding times (s)                                  
                          gringo     idlv NaGG-Gringo NaGG-IDLV   
instance                                                          
S1                        542.81   258.73      146.48    106.93  \
S2                       8973.74  2832.81     9055.40   2899.45   
S3-T4                    1032.97   991.30      619.26    616.79   
S3-T6                    1281.53  1263.10      603.44    599.17   
S3-T8                    1406.83  1399.49      688.27    688.21   
S4-T4                    1240.91   967.78      670.88    672.25   
S4-T6                    1435.00  1228.32      746.25    744.59   
S4-T8                    1522.46  1430.14      795.49    795.18   

         avg grounding sizes (KiB)                                       
                            gringo         idlv  NaGG-Gringo NaGG-IDLV   
instance                                                                 
S1                         8169.82      3785.99       345.04     15.30  \
S2                     25719681.81   1548681.74  20267666.67   7370.35   
S3-T4                    776929.01  14285843.38        19.74     19.86   
S3-T6                    389353.08  11407326.85        11.54     12.07   
S3-T8                       657.89   8437925.13        10.24     10.60   
S4-T4                   5130167.63   9440783.35         5.40      5.60   
S4-T6                   3098178.32   9297998.19         5.68      5.79   
S4-T8                   1506936.22  10549794.18         5.70      5.76   

         solved (\%)                                  
              gringo      idlv NaGG-Gringo NaGG-IDLV  
instance                                              
S1            0.5200  1.000000      1.0000   1.00000  
S2            0.1000  0.344444      0.1000   0.37037  
S3-T4         0.2900  0.342500      0.5350   0.53750  
S3-T6         0.1625  0.185000      0.4950   0.49250  
S3-T8         0.1225  0.125000      0.4375   0.42500  
S4-T4         0.1200  0.192500      0.3050   0.30500  
S4-T6         0.0475  0.080000      0.1700   0.16500  
S4-T8         0.0325  0.050000      0.1400   0.14250

In [221]:
my_latex = frame.to_latex(float_format="%.1f")
print(my_latex)

\begin{tabular}{lrrrrrrrrrrrrrrrrr}
\toprule
 & instances & \multicolumn{4}{r}{median total times (s)} & \multicolumn{4}{r}{median grounding times (s)} & \multicolumn{4}{r}{median grounding sizes (KiB)} & \multicolumn{4}{r}{solved (\%)} \\
 & instances & gringo & idlv & NaGG-Gringo & NaGG-IDLV & gringo & idlv & NaGG-Gringo & NaGG-IDLV & gringo & idlv & NaGG-Gringo & NaGG-IDLV & gringo & idlv & NaGG-Gringo & NaGG-IDLV \\
instance &  &  &  &  &  &  &  &  &  &  &  &  &  &  &  &  &  \\
\midrule
S1 & 50 & 652.2 & 216.0 & 130.3 & 90.0 & 272.0 & 129.1 & 97.9 & 78.7 & 3903.6 & 1802.2 & 172.0 & 9.7 & 0.5 & 1.0 & 1.0 & 1.0 \\
S2 & 270 & 10800.0 & 10800.0 & 10800.0 & 10800.0 & 10800.0 & 382.1 & 10800.0 & 1323.8 & 32000000.0 & 5619.5 & 32000000.0 & 4502.3 & 0.1 & 0.3 & 0.1 & 0.4 \\
S3-T4 & 400 & 1800.0 & 1800.0 & 202.6 & 204.7 & 1800.0 & 1800.0 & 186.1 & 182.0 & 2.0 & 19129.7 & 6.8 & 6.8 & 0.3 & 0.3 & 0.5 & 0.5 \\
S3-T6 & 400 & 1800.0 & 1800.0 & 264.7 & 265.5 & 1800.0 & 1800.0 & 256.4 & 247.9 & 0.

In [216]:
h1 = ["solved (\%)" for i in range(4)]
median_total_times_h1_header = ["median total times (s)" for i in range(4)] 
median_grounding_times_h1_header = ["median grounding times (s)" for i in range(4)] 
median_grounding_sizes_h1_header = ["median grounding sizes (KiB)" for i in range(4)] 

totals_h2_header = [label_converter[grounder_types[i]] for i in range(4)]
solved_h2 = [label_converter[grounder_types[i]] for i in range(4)]

h1_ = ["instances"] + median_total_times_h1_header +\
    median_grounding_times_h1_header +\
    median_grounding_sizes_h1_header +\
    h1
    
h2_ = ["instances"] + (totals_h2_header * 3) + solved_h2

columns = [h1_,h2_]

#frame = pd.DataFrame(list(zip(instance_list, size_list, *grounded_list, *solved_list)))

frame = pd.DataFrame(list(zip(instance_list, size_list, *median_tt_list, *median_gt_list, *median_gs_list,  *solved_list)))
frame.columns = ["instance"] + h2_
frame = frame.set_index("instance")
frame.columns = columns

In [217]:
frame

instances median total times (s)                                   
         instances                 gringo      idlv NaGG-Gringo NaGG-IDLV   
instance                                                                    
S1              50                 652.22    216.04      130.34     90.00  \
S2             270               10800.00  10800.00    10800.00  10800.00   
S3-T4          400                1800.00   1800.00      202.62    204.70   
S3-T6          400                1800.00   1800.00      264.74    265.51   
S3-T8          400                1800.00   1800.00      349.45    379.21   
S4-T4          400                1800.00   1800.00      292.46    288.99   
S4-T6          400                1800.00   1800.00      383.35    382.02   
S4-T8          400                1800.00   1800.00      458.57    475.29   

         median grounding times (s)                                  
                             gringo     idlv NaGG-Gringo NaGG-IDLV   
instance                                                             
S1                           272.02   129.05       97.91     78.68  \
S2                         10800.00   382.08    10800.00   1323.84   
S3-T4                       1800.00  1800.00      186.10    181.96   
S3-T6                       1800.00  1800.00      256.45    247.92   
S3-T8                       1800.00  1800.00      344.98    341.40   
S4-T4                       1800.00  1416.23      290.70    286.85   
S4-T6                       1800.00  1800.00      382.10    380.71   
S4-T8                       1800.00  1800.00      454.44    474.31   

         median grounding sizes (KiB)                                    
                               gringo      idlv  NaGG-Gringo NaGG-IDLV   
instance                                                                 
S1                            3903.55   1802.25       172.04      9.70  \
S2                        32000000.00   5619.51  32000000.00   4502.27   
S3-T4                            2.02  19129.72         6.83      6.79   
S3-T6                            0.39  20238.86         6.17      6.13   
S3-T8                            0.17  19605.33         5.90      6.12   
S4-T4                            7.35  15032.30         3.37      3.49   
S4-T6                            0.42  20065.62         3.51      3.78   
S4-T8                            0.20  20219.73         3.76      3.95   

         solved (\%)                                  
              gringo      idlv NaGG-Gringo NaGG-IDLV  
instance                                              
S1            0.5200  1.000000      1.0000   1.00000  
S2            0.1000  0.344444      0.1000   0.37037  
S3-T4         0.2900  0.342500      0.5350   0.53750  
S3-T6         0.1625  0.185000      0.4950   0.49250  
S3-T8         0.1225  0.125000      0.4375   0.42500  
S4-T4         0.1200  0.192500      0.3050   0.30500  
S4-T6         0.0475  0.080000      0.1700   0.16500  
S4-T8         0.0325  0.050000      0.1400   0.14250

In [220]:
my_latex = frame.to_latex(float_format="%.1f")
print(my_latex)

\begin{tabular}{lrrrrrrrrrrrrrrrrr}
\toprule
 & instances & \multicolumn{4}{r}{median total times (s)} & \multicolumn{4}{r}{median grounding times (s)} & \multicolumn{4}{r}{median grounding sizes (KiB)} & \multicolumn{4}{r}{solved (\%)} \\
 & instances & gringo & idlv & NaGG-Gringo & NaGG-IDLV & gringo & idlv & NaGG-Gringo & NaGG-IDLV & gringo & idlv & NaGG-Gringo & NaGG-IDLV & gringo & idlv & NaGG-Gringo & NaGG-IDLV \\
instance &  &  &  &  &  &  &  &  &  &  &  &  &  &  &  &  &  \\
\midrule
S1 & 50 & 652.2 & 216.0 & 130.3 & 90.0 & 272.0 & 129.1 & 97.9 & 78.7 & 3903.6 & 1802.2 & 172.0 & 9.7 & 0.5 & 1.0 & 1.0 & 1.0 \\
S2 & 270 & 10800.0 & 10800.0 & 10800.0 & 10800.0 & 10800.0 & 382.1 & 10800.0 & 1323.8 & 32000000.0 & 5619.5 & 32000000.0 & 4502.3 & 0.1 & 0.3 & 0.1 & 0.4 \\
S3-T4 & 400 & 1800.0 & 1800.0 & 202.6 & 204.7 & 1800.0 & 1800.0 & 186.1 & 182.0 & 2.0 & 19129.7 & 6.8 & 6.8 & 0.3 & 0.3 & 0.5 & 0.5 \\
S3-T6 & 400 & 1800.0 & 1800.0 & 264.7 & 265.5 & 1800.0 & 1800.0 & 256.4 & 247.9 & 0.